## 0) imports

In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

import pandas as pd
from sklearn.model_selection import train_test_split
from text_enrichment import *

seed = 69
OUTPUT_PATH = os.path.join(parent_dir, "2_hashtag_enrichment", "output")
PROJ_PATH = "/home/francesco/Desktop/IRONY/main_22-12/NLP_irony_detector/"

Importato package text_enrichment.


## 1) loading datasets

In [2]:
from utils import load_datasets
previous_folder = "1_data_exploration"
train_set, val_set, test_set = load_datasets(os.path.join(parent_dir, previous_folder, "output"))

## 2) preprocessing with placeholders
this part allows to do text enrichment considering also placeholders as features

In [5]:
placeholder_list = []


## 3) hastag text enrichment
finding
$$
    P(\text{iro}=1|\text{feature}) =
    \frac{|\text{tweets\_iro\_with\_feature}|}
    {|\text{tweets\_iro\_with\_feature}| + |\text{tweets\_non\_iro\_with\_feature}|}
$$
for a more detailed explanation go to text_enrichment/main_text_enrichment.

this method does not take into account co occurrences (also dataset is too tiny to implement it).


In [6]:
prob_of_features = find_relevant_features(train_set)

the new choosen feature is prob: the idea behind it is to fed it into the classifier part of the model with the desired effect of increase the mean activations of the classifier.

In [7]:
train_set['prob'] = train_set['text'].apply(lambda x : get_prob_from_sentence(x, prob_of_features))
val_set['prob'] = val_set['text'].apply(lambda x : get_prob_from_sentence(x, prob_of_features))
test_set['prob'] = test_set['text'].apply(lambda x : get_prob_from_sentence(x, prob_of_features))

In [8]:
train_set.tail()

,iro,text,prob
7397,1,"che ci frega di mario monti, noi abbiamo mario...",0.000000
7398,1,Strepitoso il titolo in prima di Libero sul go...,0.000000
7399,1,"@nataliacavalli Consolati, il governo #Monti h...",0.188034
7400,1,"@SheisCandida beh, beate loro! Io nn possiedo ...",0.000000
7401,1,"Caro #Renzi,se #Grillo spaccava i computer e o...",0.048708


## 3.1) percentages

In [9]:
train_len = len(train_set)
val_len = len(val_set)
test_len = len(test_set)

n_considered_tweets_train  = sum(sum([train_set['prob']!=0]))
n_considered_tweets_val  = sum(sum([val_set['prob']!=0]))
n_considered_tweets_test  = sum(sum([test_set['prob']!=0]))

In [10]:
print(f"train {n_considered_tweets_train/train_len}")
print(f"val {n_considered_tweets_val/val_len}")
print(f"test {n_considered_tweets_test/test_len}")

train 0.27614158335584976
val 0.009
test 0.004


The considered features in our dataset are unbalanced. to make this algorithm work we choosed to merge and shuffle the tweets of all datasets

## 4) create dataframe

In [11]:
# not here bec we are using shuffled data!

# train_set.to_csv(OUTPUT_PATH+"train.csv", index=False)
# val_set.to_csv(OUTPUT_PATH+"val.csv", index=False)
# test_set.to_csv(OUTPUT_PATH+"test.csv", index=False)

## 5) trying to improve the val test percentages

In [12]:
dataset = pd.concat([train_set, val_set, test_set])
dataset.drop(columns='prob', inplace=True)
dataset.shape

(9402, 2)

In [13]:
dataset.head()

,iro,text
0,0,Intanto la partita per Via Nazionale si compli...
1,0,"False illusioni, sgradevoli realtà Mario Monti..."
2,0,"False illusioni, sgradevoli realtà #editoriale..."
3,0,Mario Monti: Berlusconi risparmi all'Italia il...
4,0,Mario Monti: Berlusconi risparmi all'Italia il...


In [14]:
train_set, test_set = train_test_split(dataset, test_size=0.2125, shuffle=True, random_state=seed)
test_set, val_set = train_test_split(test_set, test_size=0.5, shuffle=True, random_state=seed)

print("I'm keeping the sqme ratio")
print(f"train shape: {train_set.shape}")
print(f"val shape: {val_set.shape}")
print(f"test shape: {test_set.shape}")

I'm keeping the sqme ratio
train shape: (7404, 2)
val shape: (999, 2)
test shape: (999, 2)


## 6) preprocessing

## 7) hastag enfirhment

In [15]:
prob_of_features = find_relevant_features(train_set)

train_set['prob'] = train_set['text'].apply(lambda x : get_prob_from_sentence(x, prob_of_features))
val_set['prob'] = val_set['text'].apply(lambda x : get_prob_from_sentence(x, prob_of_features))
test_set['prob'] = test_set['text'].apply(lambda x : get_prob_from_sentence(x, prob_of_features))

In [16]:
train_set.tail()

,iro,text,prob
823,0,'Una buona scuola punitiva e non democratica n...,0.000000
4374,0,è tutto un gioco di poteri e noi coglioni ad a...,0.096154
229,0,Linee guida 'la buona scuola': cosa prevedono ...,0.000000
4041,1,Di #Grillo apprezzo la profondità di pensiero ...,0.053218
7371,0,Ma il #palermo dove li mette tutti sti acquisti??,0.000000


## 7.1) percentages

In [17]:
train_len = len(train_set)
val_len = len(val_set)
test_len = len(test_set)

n_considered_tweets_train  = sum(sum([train_set['prob']!=0]))
n_considered_tweets_val  = sum(sum([val_set['prob']!=0]))
n_considered_tweets_test  = sum(sum([test_set['prob']!=0]))

In [18]:
print(f"train {n_considered_tweets_train/train_len}")
print(f"val {n_considered_tweets_val/val_len}")
print(f"test {n_considered_tweets_test/test_len}")

train 0.3117233927606699
val 0.3133133133133133
test 0.33433433433433435


## 8) create dataframe

In [19]:
train_set.to_csv(OUTPUT_PATH+ "/train.csv", index=False)
val_set.to_csv(OUTPUT_PATH + "/val.csv", index=False)
test_set.to_csv(OUTPUT_PATH + "/test.csv", index=False)